In [1]:
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import zipfile
import os
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional,LSTM,GRU,Dense
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn import metrics
import nltk
from nltk.tokenize import word_tokenize
import warnings
import tensorflow as tf
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
dataset = pd.read_csv('train.txt')

In [3]:
dataset

,i didnt feel humiliated;sadness
0,i can go from feeling so hopeless to so damned...
1,im grabbing a minute to post i feel greedy wro...
2,i am ever feeling nostalgic about the fireplac...
3,i am feeling grouchy;anger
4,ive been feeling a little burdened lately wasn...
...,...
15994,i just had a very brief time in the beanbag an...
15995,i am now turning and i feel pathetic that i am...
15996,i feel strong and good overall;joy
15997,i feel like this was such a rude comment and i...


In [4]:
f=open('train.txt','r')
x_train=[]
y_train=[]
for i in f:
    l=i.split(';')
    y_train.append(l[1].strip())
    x_train.append(l[0])
f=open('test.txt','r')
x_test=[]
y_test=[]
for i in f:
    l=i.split(';')
    y_test.append(l[1].strip())
    x_test.append(l[0])
data_train=pd.DataFrame({'Text':x_train,'Emotion':y_train})
data_test=pd.DataFrame({'Text':x_test,'Emotion':y_test})
data=data_train.append(data_test,ignore_index=True)

In [5]:
data

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
17995,i just keep feeling like someone is being unki...,anger
17996,im feeling a little cranky negative after this...,anger
17997,i feel that i am useful to my people and that ...,joy
17998,im feeling more comfortable with derby i feel ...,joy


In [6]:
def clean_data(data):
    
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)

    # Remove new line characters
    data = re.sub('\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub("\'", "", data)
    
    data=re.sub("(\\d|\\W)+"," ",data)
    data = re.sub(r'[?|$|.|!]',r'',data)
    data = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", data)
    
    #data = re.sub('[^A-Za-z0-9]+', '', data)
        
    return data

In [7]:
def process_message(message, lower_case = True, stem = True, stop_words = True, gram = 2):
    if lower_case:
        message = message.lower()
    words = word_tokenize(message)
    words = [w for w in words if len(w) > 2]
    if gram > 1:
        w = []
        for i in range(len(words) - gram + 1):
            w += [' '.join(words[i:i + gram])]
        return w
    if stop_words:
        sw = stopwords.words('english')
        words = [word for word in words if word not in sw]
    if stem:
        stemmer = PorterStemmer()
        words = [stemmer.stem(word) for word in words]   
    return words

In [8]:
texts=[''.join(clean_data(text)) for text in data.Text]
texts_train=[''.join(clean_data(text)) for text in x_train]
texts_test=[''.join(clean_data(text)) for text in x_test]

In [9]:
print(texts[:10])

['i didnt feel humiliated', 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake', 'im grabbing a minute to post i feel greedy wrong', 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property', 'i am feeling grouchy', 'ive been feeling a little burdened lately wasnt sure why that was', 'ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny', 'i feel as confused about life as a teenager or as jaded as a year old man', 'i have been with petronas for years i feel that petronas has performed well and made a huge profit', 'i feel romantic too']


In [10]:
print(texts_train[:10])

['i didnt feel humiliated', 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake', 'im grabbing a minute to post i feel greedy wrong', 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property', 'i am feeling grouchy', 'ive been feeling a little burdened lately wasnt sure why that was', 'ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny', 'i feel as confused about life as a teenager or as jaded as a year old man', 'i have been with petronas for years i feel that petronas has performed well and made a huge profit', 'i feel romantic too']


In [11]:
print(texts_test[:10])

['im feeling rather rotten so im not very ambitious right now', 'im updating my blog because i feel shitty', 'i never make her separate from me because i don t ever want her to feel like i m ashamed with her', 'i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived', 'i was feeling a little vain when i did this one', 'i cant walk into a shop anywhere where i do not feel uncomfortable', 'i felt anger when at the end of a telephone call', 'i explain why i clung to a relationship with a boy who was in many ways immature and uncommitted despite the excitement i should have been feeling for getting accepted into the masters program at the university of virginia', 'i like to have the same breathless feeling as a reader eager to see what will happen next', 'i jest i feel grumpy tired and pre menstrual which i probably am but then again its only been a week and im about as fit as a walrus on vacation for the summer']


In [12]:
texts=[' '.join(process_message(text)) for text in data.Text]
texts_train=[' '.join(process_message(text)) for text in x_train]
texts_test=[' '.join(process_message(text)) for text in x_test]

In [13]:
print(texts[:10])

['didnt feel feel humiliated', 'can from from feeling feeling hopeless hopeless damned damned hopeful hopeful just just from from being being around around someone someone who who cares cares and and awake', 'grabbing minute minute post post feel feel greedy greedy wrong', 'ever feeling feeling nostalgic nostalgic about about the the fireplace fireplace will will know know that that still still the the property', 'feeling grouchy', 'ive been been feeling feeling little little burdened burdened lately lately wasnt wasnt sure sure why why that that was', 'ive been been taking taking milligrams milligrams times times recommended recommended amount amount and and ive ive fallen fallen asleep asleep lot lot faster faster but but also also feel feel like like funny', 'feel confused confused about about life life teenager teenager jaded jaded year year old old man', 'have been been with with petronas petronas for for years years feel feel that that petronas petronas has has performed performe

In [14]:
print(texts_train[:10])

['didnt feel feel humiliated', 'can from from feeling feeling hopeless hopeless damned damned hopeful hopeful just just from from being being around around someone someone who who cares cares and and awake', 'grabbing minute minute post post feel feel greedy greedy wrong', 'ever feeling feeling nostalgic nostalgic about about the the fireplace fireplace will will know know that that still still the the property', 'feeling grouchy', 'ive been been feeling feeling little little burdened burdened lately lately wasnt wasnt sure sure why why that that was', 'ive been been taking taking milligrams milligrams times times recommended recommended amount amount and and ive ive fallen fallen asleep asleep lot lot faster faster but but also also feel feel like like funny', 'feel confused confused about about life life teenager teenager jaded jaded year year old old man', 'have been been with with petronas petronas for for years years feel feel that that petronas petronas has has performed performe

In [15]:
print(texts_test[:10])

['feeling rather rather rotten rotten not not very very ambitious ambitious right right now', 'updating blog blog because because feel feel shitty', 'never make make her her separate separate from from because because don don ever ever want want her her feel feel like like ashamed ashamed with with her', 'left with with bouquet bouquet red red and and yellow yellow tulips tulips under under arm arm feeling feeling slightly slightly more more optimistic optimistic than than when when arrived', 'was feeling feeling little little vain vain when when did did this this one', 'cant walk walk into into shop shop anywhere anywhere where where not not feel feel uncomfortable', 'felt anger anger when when the the end end telephone telephone call', 'explain why why clung clung relationship relationship with with boy boy who who was was many many ways ways immature immature and and uncommitted uncommitted despite despite the the excitement excitement should should have have been been feeling feeli

In [16]:
class_names=['anger','sadness','fear','joy','surprise','love']

In [21]:
from sklearn.svm import SVC
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', SVC(kernel='rbf'))])

text_clf.fit(x_train, y_train)

svcpredicted = text_clf.predict(x_test)

In [22]:
print(svcpredicted)

['sadness' 'sadness' 'sadness' ... 'joy' 'joy' 'fear']


In [23]:
svc1result = text_clf.predict(['i think it s the easiest time of year to feel dissatisfied'])
print(svc1result)

['anger']
